In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import warnings
from sklearn.utils import shuffle 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
%matplotlib inline
warnings.filterwarnings('ignore')

## Data Preprocessing 

In [2]:
df = pd.read_csv("data/wine.csv")

In [3]:
df.head()

,Wine,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [4]:
df_y = df['Wine'].astype("str")
df_Y = pd.get_dummies(df_y)
df_Y.columns = ['Wine_1', "Wine_2", "Wine_3"]
df_Y.head()

,Wine_1,Wine_2,Wine_3
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0


In [5]:
df_X = df.drop(columns=["Wine"])
df_X.head()

,Alcohol,Malic.acid,Ash,Acl,Mg,Phenols,Flavanoids,Nonflavanoid.phenols,Proanth,Color.int,Hue,OD,Proline
0,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [6]:
df_X, df_Y = shuffle(df_X, df_Y)
std = StandardScaler()
df_X = std.fit_transform(df_X)

In [7]:
X_train, X_val, Y_train, Y_val = train_test_split(df_X, df_Y,
                                                  test_size=0.2,
                                                  random_state=0)

##  Construct Model 

In [8]:
# 单层的全连接神经网络

In [9]:
def Model(X, ow, b1):
    a0 = X
    a1 = (tf.matmul(ow, a0) + b1)
    output = tf.nn.softmax(a1)
    return output

In [10]:
X = tf.placeholder(dtype="float", shape=[13, None], name="X")
Y = tf.placeholder(dtype="float", shape=[3, None], name="Y")
ow = tf.get_variable(shape=[3, 13], initializer=tf.contrib.layers.xavier_initializer(), name='ow')
b1 = tf.Variable(tf.zeros([3, 1]))
output = Model(X, ow, b1)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Colocations handled automatically by placer.


In [21]:
# loss
cross_entropy = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(output),
                                              reduction_indices=[0]))
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

In [22]:
feed_dict_train = {X: X_train.T,
                   Y: Y_train.T}
feed_dict_val = {X: X_val.T,
                 Y: Y_val.T}

In [44]:
# Train
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epcho in range(500):
        _, loss, y_pred = sess.run([train_step, cross_entropy, output], feed_dict=feed_dict_train)
        accuracy = (np.sum((np.argmax(y_pred, 0) == np.argmax(Y_train.values.T, 0))))/Y_train.shape[0]
        
        loss_val = cross_entropy.eval(feed_dict=feed_dict_val)
        y_pred_val = output.eval(feed_dict=feed_dict_val)
        accuaray_val = (np.sum((np.argmax(y_pred_val, 0) == np.argmax(Y_val.values.T, 0))))/Y_val.shape[0]
        if epcho % 10 == 0:
            print("Train loss:", loss,
                  "Train accuary:", accuracy)
            print("Val loss:", loss_val,
                  "Val accuary:", accuaray_val)
            print("----------------------------")

Train loss: 6.2170796 Train accuary: 0.3380281690140845
Val loss: 4.259977 Val accuary: 0.5833333333333334
----------------------------
Train loss: 5.9380155 Train accuary: 0.352112676056338
Val loss: 4.1068196 Val accuary: 0.6111111111111112
----------------------------
Train loss: 5.686493 Train accuary: 0.3732394366197183
Val loss: 3.9691794 Val accuary: 0.6111111111111112
----------------------------
Train loss: 5.461677 Train accuary: 0.4014084507042254
Val loss: 3.8448818 Val accuary: 0.6111111111111112
----------------------------
Train loss: 5.2637463 Train accuary: 0.4295774647887324
Val loss: 3.7329898 Val accuary: 0.5833333333333334
----------------------------
Train loss: 5.0932364 Train accuary: 0.49295774647887325
Val loss: 3.6333034 Val accuary: 0.6111111111111112
----------------------------
Train loss: 4.9500775 Train accuary: 0.5774647887323944
Val loss: 3.545863 Val accuary: 0.6388888888888888
----------------------------
Train loss: 4.8328032 Train accuary: 0.654929